## Import packages and create custom functions

In [11]:
import boto3
import numpy as np
import pandas as pd

In [15]:
def remove_unnamed_columns(df: pd.DataFrame):
    '''
    Create list of columns names without starintng with 'Unnamed...'
    :param df: pandas dataframe
    :return important_cols: list of column names
    '''
    cols = df.columns
    important_cols = []

    for col in cols:
        if col[:7] != 'Unnamed':
            important_cols.append(col)

    return important_cols

## Prepare data

In [4]:
# Connect with AWS S3
s3_client = boto3.client('s3')
resposne = s3_client.get_object(Bucket='football-bookmaker-storage', Key='matches.csv')

# Create dataframe based on matches.csv
df = pd.read_csv(resposne.get("Body"))

In [16]:
# Remove unnecessary columns and rows
columns = remove_unnamed_columns(df)
df = df[columns]
df = df[(df['who_win'] != 'err') & (df['away_passes_percent'] != '-1')]

In [6]:
teams = list(df['home'].unique())

In [26]:
# Create column with winner team
conditions = [
    df['who_win'] == 'home',
    df['who_win'] == 'away',
    df['who_win'] == 'draw'
]
choices = [df['home'], df['away'], 'draw']
df['winner_team'] = np.select(conditions, choices)

# Create round as integer
df['round_int'] = df['round'].str[16:].astype(int)
df = df[['date', 'season', 'round_int', 'home', 'away', 'winner_team']]

df.head()

,date,season,round_int,home,away,winner_team
0,2019-08-09T19:00:00+00:00,2019,1,Liverpool,Norwich,Liverpool
1,2019-08-10T11:30:00+00:00,2019,1,West Ham,Manchester City,Manchester City
2,2019-08-10T14:00:00+00:00,2019,1,Bournemouth,Sheffield Utd,draw
3,2019-08-10T14:00:00+00:00,2019,1,Burnley,Southampton,Burnley
4,2019-08-10T14:00:00+00:00,2019,1,Crystal Palace,Everton,draw


In [51]:
data = {}
seasons = df['season'].unique()

season = 2019
team = 'Norwich'

df_team = df.loc[(df['season'] == season) & \
                 ((df['home'] == team) | (df['away'] == team))]

df_team

# for i, row in df_team.iterrows():
#     print(df_team.iloc[i, :])
#     print(df_team.iloc[i+1, :])
#     break


# for season in seasons:
#     teams = df ['home'].unique()
#     print(teams)
#     break
    # for team in teams:
    #     df.loc[(df['winner_team'] == 'Liverpool') & (df['season'] == 2021)]

,date,season,round_int,home,away,winner_team
0,2019-08-09T19:00:00+00:00,2019,1,Liverpool,Norwich,Liverpool
16,2019-08-17T14:00:00+00:00,2019,2,Norwich,Newcastle,Norwich
25,2019-08-24T11:30:00+00:00,2019,3,Norwich,Chelsea,Chelsea
39,2019-08-31T14:00:00+00:00,2019,4,West Ham,Norwich,West Ham
45,2019-09-14T16:30:00+00:00,2019,5,Norwich,Manchester City,Norwich
51,2019-09-21T14:00:00+00:00,2019,6,Burnley,Norwich,Burnley
63,2019-09-28T14:00:00+00:00,2019,7,Crystal Palace,Norwich,Crystal Palace
76,2019-10-05T14:00:00+00:00,2019,8,Norwich,Aston Villa,Aston Villa
80,2019-10-19T14:00:00+00:00,2019,9,Bournemouth,Norwich,draw
96,2019-10-27T15:30:00+00:00,2019,10,Norwich,Manchester United,Manchester United


In [38]:
len(df.loc[(df['winner_team'] == 'Liverpool') & (df['season'] == 2021)])

28